# Onset Confidence Intervals

In [1]:
import pickle
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pwmv_ecog.analysis.image_preference import ImagePreferenceAnalysis
from pwmv_ecog.analysis.mean_roi import MeanROIAnalysis as mra
from pwmv_ecog.manuscript.nature_figures import SIGBARCOLORS, SUBJECTS

In [2]:
resultsOfInterest = {'mra': [("EVC","Post>Pre"),("HLVC", "Post>Pre"),("FPN","Post>Pre")], 'ipa': [("HLVC","Post>0"),("EVC","Post>0")]}
expectedN = 2000

In [3]:
meanROIReal: mra = mra(SUBJECTS).load()
imagePrefReal: ImagePreferenceAnalysis = ImagePreferenceAnalysis(SUBJECTS).load()
imagePrefTypeTypeMap = {'Pre>0':'Pre','Post>0':'Post','Post>Pre':'Post - Pre'}

Loaded MeanROIAnalysis from ../Analysis/MeanROI/network/MeanROIAnalysis_network_HGP_1001,2-6,9-21.pkl.
Loaded ImagePreferenceAnalysis from ../Analysis/ImagePref/network/ImagePreferenceAnalysis_HGP_network_None_0.1_1001,2-6,9-21.pkl.


In [4]:
bootstrapSaveDir  = Path("../Analysis/MeanROI/data/bootstrap")
bootstrapResultFiles = sorted(bootstrapSaveDir.glob('*electrode_mean*bootstrap_results.pkl'))
print(f'Found {len(bootstrapResultFiles)} MRA bootstrap results.')

def filenameToMRACondName(fname):
    return tuple(fname.name.removesuffix('_bootstrap_results.pkl').replace(f'_electrode_mean_HGP_{SUBJECTS}','').split('_'))

mraBootResults = {}
realMRAClusterStats = {}
for file in bootstrapResultFiles:
    conditionName = filenameToMRACondName(file)
    if conditionName not in resultsOfInterest['mra']:
        continue
    with file.open('rb') as f:
        mraBootResults[conditionName] = pickle.load(f)
    realMRAClusterStats[conditionName] = meanROIReal.clusterStats[conditionName[0]][conditionName[1]]
assert all([len(br)==expectedN for br in mraBootResults.values()])
assert len(mraBootResults) == len(resultsOfInterest['mra'])
mraBootResults = {k:mraBootResults[k] for k in resultsOfInterest['mra']}
print(f'Found {len(mraBootResults)} conditions of interest for Mean ROI Analysis all with {expectedN} bootstrapped replications: {list(mraBootResults.keys())}')

Found 11 MRA bootstrap results.
Found 3 conditions of interest for Mean ROI Analysis all with 2000 bootstrapped replications: [('EVC', 'Post>Pre'), ('HLVC', 'Post>Pre'), ('FPN', 'Post>Pre')]


In [5]:
imagePrefDataPath = Path('../Analysis/ImagePref/data/bootstrap')
imagePrefPickles = list(imagePrefDataPath.glob('**/*bootstrap_results.pkl'))
print(f'Found {len(imagePrefPickles)} IPA bootstrap results.')
imagePrefPickles

def filenameToIPACondName(fname):
    return tuple(fname.name.removesuffix('_bootstrap_results.pkl').replace(f'_image_preference_HGP_{SUBJECTS}','').split('_'))

ipaBootResults = {}
realIPAClusterStats = {}
for file in imagePrefPickles:
    conditionName = filenameToIPACondName(file)
    if conditionName not in resultsOfInterest['ipa']:
        continue
    # print(conditionName)
    with file.open('rb') as f:
        ipaBootResults[conditionName]=pickle.load(f)
    realIPAClusterStats[conditionName] = imagePrefReal.clusterStats[conditionName[0]][imagePrefTypeTypeMap[conditionName[1]]]
print([(k,len(v)) for k,v in ipaBootResults.items()])
print('Trimming superfluous bootstraps...')
ipaBootResults = {k:v[:expectedN] for k,v in ipaBootResults.items()}
print([(k,len(v)) for k,v in ipaBootResults.items()])

Found 4 IPA bootstrap results.
[(('EVC', 'Post>0'), 2026), (('HLVC', 'Post>0'), 2126)]
Trimming superfluous bootstraps...
[(('EVC', 'Post>0'), 2000), (('HLVC', 'Post>0'), 2000)]


In [6]:
def matchedOnsets(bootResults, realClusterStats):
    matchedOnsets = {}
    for condName,bootResults in bootResults.items():
        sigClusters = realClusterStats[condName].clusterDF()[realClusterStats[condName].clusterDF().pVal <=0.05]
        matchedOnsets[condName] = []
        for realClusterRow in sigClusters.itertuples():
            onsets = []
            for results in bootResults:
                if pd.isna(results.sigClusterPlotPoints).all():
                    onsets.append(np.nan)
                else:
                    overlapBootClusters = results.clusterDF()[(results.clusterDF().endCluster>realClusterRow.startCluster) & (results.clusterDF().startCluster <realClusterRow.endCluster) & (results.clusterDF().pVal <=0.05)]
                    if len(overlapBootClusters):
                        onsets.append(results.refIndex[overlapBootClusters['startCluster'].sort_values().iat[0]])
                    else:
                        onsets.append(np.nan)
            matchedOnsets[condName].append(onsets)
    return matchedOnsets
mraOnsets = matchedOnsets(mraBootResults, realMRAClusterStats)
ipaOnsets = matchedOnsets(ipaBootResults, realIPAClusterStats)

## 95% CI

In [7]:
onsetCI = {}
for analysisName, onsets in [('mra',mraOnsets), ('ipa',ipaOnsets)]:
    for condName, onsetList in onsets.items():
        onsetCI[analysisName, condName] = np.nanpercentile(onsetList, [2.5, 97.5], method="higher")

In [8]:
print('95% CI')
onsetCI

95% CI


{('mra', ('EVC', 'Post>Pre')): array([0.74414062, 0.89257812]),
 ('mra', ('HLVC', 'Post>Pre')): array([0.19140625, 0.55078125]),
 ('mra', ('FPN', 'Post>Pre')): array([0.609375  , 0.94335938]),
 ('ipa', ('EVC', 'Post>0')): array([0.2421875 , 0.45507812]),
 ('ipa', ('HLVC', 'Post>0')): array([0.15234375, 0.41992188])}